# Building classification model on CIFAR-10 multiclass dataset
It is based on the convolutional neural network implemented with PyTorch API.

In [39]:
## Packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## Device configuration and defining hyper parameters

In [71]:
# I am using GPU acceleration :)
device = torch.device('cuda')

# Hyper parameters
num_epochs = 10
batch_size = 40
learning_rate = 0.001

## Loading data

In [72]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [73]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [74]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [75]:
train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [76]:
test_dataset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [77]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [78]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [79]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Building CNN Model

In [80]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #(in_channels, out_channels, kernel_size)
        self.pool = nn.MaxPool2d(2, 2) #
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x   

In [81]:
# Model instantiation
model = ConvNet().to(device)

In [82]:
# Setting Loss function
criterion = nn.CrossEntropyLoss()

# Setting optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training Loop

In [83]:
n_total_steps = len(train_loader) # number of training mini-batches
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # mini-batch is sent to the device 
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 500 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step: {i+1}/{n_total_steps}, Loss: {loss.item():.4f}')

print('Training Finished. Hurraay!')

Epoch: 1/10, Step: 500/1250, Loss: 1.6062
Epoch: 1/10, Step: 1000/1250, Loss: 1.3311
Epoch: 2/10, Step: 500/1250, Loss: 1.5644
Epoch: 2/10, Step: 1000/1250, Loss: 1.1216
Epoch: 3/10, Step: 500/1250, Loss: 1.3293
Epoch: 3/10, Step: 1000/1250, Loss: 1.4173
Epoch: 4/10, Step: 500/1250, Loss: 1.0742
Epoch: 4/10, Step: 1000/1250, Loss: 1.3378
Epoch: 5/10, Step: 500/1250, Loss: 1.0384
Epoch: 5/10, Step: 1000/1250, Loss: 1.1379
Epoch: 6/10, Step: 500/1250, Loss: 1.3039
Epoch: 6/10, Step: 1000/1250, Loss: 0.9081
Epoch: 7/10, Step: 500/1250, Loss: 1.0259
Epoch: 7/10, Step: 1000/1250, Loss: 0.8653
Epoch: 8/10, Step: 500/1250, Loss: 1.0190
Epoch: 8/10, Step: 1000/1250, Loss: 1.0630
Epoch: 9/10, Step: 500/1250, Loss: 0.4741
Epoch: 9/10, Step: 1000/1250, Loss: 0.8329
Epoch: 10/10, Step: 500/1250, Loss: 1.1568
Epoch: 10/10, Step: 1000/1250, Loss: 0.9841
Training Finished. Hurraay!


## TEST

In [84]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc}%')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc}%')
                

Accuracy of the network: 63.67%
Accuracy of plane: 71.1%
Accuracy of car: 82.4%
Accuracy of bird: 55.9%
Accuracy of cat: 43.6%
Accuracy of deer: 56.7%
Accuracy of dog: 38.1%
Accuracy of frog: 73.1%
Accuracy of horse: 75.3%
Accuracy of ship: 72.4%
Accuracy of truck: 68.1%
